# Stock Price Prediction Challenge 📈 

<img src="https://miro.medium.com/v2/resize:fit:2000/1*hhq3ybwbyA3p0dWuLFtLMQ.jpeg" width="600" height="450">


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import joblib
import warnings
warnings.filterwarnings('ignore')

# 1. Load and preprocess the data
def load_and_preprocess_data(file_path):
    print("Loading and preprocessing data...")
    # Load the dataset
    df = pd.read_csv(file_path)
    
    # Remove 'Unnamed: 0' column if it exists
    if 'Unnamed: 0' in df.columns:
        df = df.drop('Unnamed: 0', axis=1)
    
    # Convert Date to datetime and set as index
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    
    # Display basic information
    print(f"Dataset shape: {df.shape}")
    print(f"Missing values:\n{df.isnull().sum()}")
    
    # Handle missing values
    # For price data, use forward fill followed by backward fill
    for col in ['Open', 'High', 'Low', 'Close', 'Adj Close']:
        df[col] = df[col].fillna(method='ffill').fillna(method='bfill')
    
    # For volume, use median
    df['Volume'] = df['Volume'].fillna(df['Volume'].median())
    
    print(f"Missing values after handling:\n{df.isnull().sum()}")
    return df
    
    # Convert Date to datetime and set as index
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    
    # Display basic information
    print(f"Dataset shape: {df.shape}")
    print(f"Missing values:\n{df.isnull().sum()}")
    
    # Handle missing values
    # For price data, use forward fill followed by backward fill
    for col in ['Open', 'High', 'Low', 'Close', 'Adj Close']:
        df[col] = df[col].fillna(method='ffill').fillna(method='bfill')
    
    # For volume, use median
    df['Volume'] = df['Volume'].fillna(df['Volume'].median())
    
    print(f"Missing values after handling:\n{df.isnull().sum()}")
    return df

# 2. Exploratory Data Analysis
def perform_eda(df):
    print("\nPerforming Exploratory Data Analysis...")
    # Plot the closing price
    plt.figure(figsize=(12, 6))
    plt.plot(df['Close'])
    plt.title('Stock Closing Price Over Time')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.grid(True)
    plt.savefig('closing_price_over_time.png')
    plt.close()
    
    # Calculate daily returns
    df['Daily_Return'] = df['Close'].pct_change()
    
    # Plot the daily returns
    plt.figure(figsize=(12, 6))
    plt.plot(df['Daily_Return'])
    plt.title('Daily Returns Over Time')
    plt.xlabel('Date')
    plt.ylabel('Return')
    plt.grid(True)
    plt.savefig('daily_returns.png')
    plt.close()
    
    # Distribution of daily returns
    plt.figure(figsize=(12, 6))
    sns.histplot(df['Daily_Return'].dropna(), kde=True)
    plt.title('Distribution of Daily Returns')
    plt.xlabel('Daily Return')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.savefig('return_distribution.png')
    plt.close()
    
    # Check for stationarity
    result = adfuller(df['Close'].dropna())
    print('Augmented Dickey-Fuller Test:')
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    
    # Correlation matrix
    correlation = df.corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation, annot=True, cmap='coolwarm')
    plt.title('Correlation Matrix')
    plt.savefig('correlation_matrix.png')
    plt.close()
    
    # Try to perform seasonal decomposition if enough data
    try:
        # Decompose the time series (using a subset if too large)
        if len(df) > 1000:
            sample_df = df[-1000:].copy()
        else:
            sample_df = df.copy()
            
        result = seasonal_decompose(sample_df['Close'], model='multiplicative', period=252)
        
        # Plot decomposition
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(12, 12))
        result.observed.plot(ax=ax1)
        ax1.set_title('Observed')
        result.trend.plot(ax=ax2)
        ax2.set_title('Trend')
        result.seasonal.plot(ax=ax3)
        ax3.set_title('Seasonality')
        result.resid.plot(ax=ax4)
        ax4.set_title('Residuals')
        plt.tight_layout()
        plt.savefig('seasonal_decomposition.png')
        plt.close()
    except Exception as e:
        print(f"Could not perform seasonal decomposition: {e}")
    
    return df
# Add this function to detect and report problematic values in your dataset
def check_for_problematic_values(df, stage_name=""):
    """Check for NaN, infinity or extremely large values in the dataframe."""
    print(f"\nChecking for problematic values at stage: {stage_name}")
    
    # Check for NaN values
    nan_count = df.isna().sum().sum()
    print(f"Total NaN values: {nan_count}")
    if nan_count > 0:
        print("Columns with NaN values:")
        print(df.isna().sum()[df.isna().sum() > 0])
    
    # Check for infinity values
    inf_count = np.isinf(df.select_dtypes(include=[np.number])).sum().sum()
    print(f"Total infinity values: {inf_count}")
    if inf_count > 0:
        inf_cols = df.columns[np.isinf(df.select_dtypes(include=[np.number])).any()]
        print(f"Columns with infinity values: {list(inf_cols)}")
    
    # Check for extremely large values
    max_values = df.select_dtypes(include=[np.number]).max()
    large_value_cols = max_values[max_values > 1e10].index.tolist()
    if large_value_cols:
        print(f"Columns with extremely large values (>1e10): {large_value_cols}")
    
    return nan_count > 0 or inf_count > 0 or bool(large_value_cols)

# Modify the engineer_features function to handle division by zero
def engineer_features(df):
    print("\nEngineering features...")
    # Moving averages
    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA10'] = df['Close'].rolling(window=10).mean()
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA50'] = df['Close'].rolling(window=50).mean()
    df['MA200'] = df['Close'].rolling(window=200).mean()
    
    # Exponential moving averages
    df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
    
    # MACD
    df['MACD'] = df['EMA12'] - df['EMA26']
    df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_Histogram'] = df['MACD'] - df['Signal_Line']
    
    # RSI - Modified to avoid division by zero
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    # Avoid division by zero by adding a small value
    rs = gain / (loss + 1e-10)  # Add small epsilon to prevent division by zero
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # Bollinger Bands - Modified to handle zero standard deviation
    df['MA20_std'] = df['Close'].rolling(window=20).std()
    # Replace zero std with a small value to avoid division by zero
    df['MA20_std'] = df['MA20_std'].replace(0, 1e-10)
    
    df['Upper_Band'] = df['MA20'] + (df['MA20_std'] * 2)
    df['Lower_Band'] = df['MA20'] - (df['MA20_std'] * 2)
    # Avoid division by zero in BB_Width
    df['BB_Width'] = (df['Upper_Band'] - df['Lower_Band']) / (df['MA20'] + 1e-10)
    
    # Momentum and Rate of Change
    df['Momentum'] = df['Close'] - df['Close'].shift(5)
    # Avoid division by zero in ROC
    df['ROC'] = df['Close'] / (df['Close'].shift(5) + 1e-10) - 1
    df['ROC'] = df['ROC'] * 100
    
    # Average True Range (ATR)
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift())
    low_close = np.abs(df['Low'] - df['Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)
    df['ATR'] = true_range.rolling(14).mean()
    
    # Lag features
    for i in range(1, 6):
        df[f'Lag_{i}'] = df['Close'].shift(i)
    
    # Price change features
    df['Price_Change'] = df['Close'] - df['Open']
    # Avoid division by zero in Price_Change_Pct
    df['Price_Change_Pct'] = (df['Close'] - df['Open']) / (df['Open'] + 1e-10) * 100
    
    # Volume indicators
    df['Volume_MA5'] = df['Volume'].rolling(window=5).mean()
    # Avoid division by zero in Volume_Change
    df['Volume_Change'] = df['Volume'] / (df['Volume'].shift(1) + 1e-10) - 1
    
    # Target variable: price 5 days ahead
    df['Target'] = df['Close'].shift(-5)
    
    # High/Low diff
    df['HL_Diff'] = df['High'] - df['Low']
    # Avoid division by zero in HL_Pct
    df['HL_Pct'] = df['HL_Diff'] / (df['Low'] + 1e-10) * 100
    
    # Day of week feature
    df['Day_of_Week'] = df.index.dayofweek
    
    # Month feature
    df['Month'] = df.index.month
    
    # Check for problems before cleaning
    has_problems = check_for_problematic_values(df, "after feature engineering, before cleaning")
    
    # Replace infinity values with NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # Drop missing values after feature creation
    df_complete = df.dropna().copy()
    
    # Optional: For features that might still have extreme values, clip them
    # For each numeric column, clip values to reasonable bounds
    # Define a function to clip values based on their distribution
    def clip_extreme_values(series, n_std=5):
        if pd.api.types.is_numeric_dtype(series):
            mean = series.mean()
            std = series.std()
            lower_bound = mean - n_std * std
            upper_bound = mean + n_std * std
            return series.clip(lower_bound, upper_bound)
        return series
    
    # Apply clipping to all numeric columns
    for col in df_complete.select_dtypes(include=[np.number]).columns:
        if col not in ['Day_of_Week', 'Month']:  # Skip categorical numeric columns
            df_complete[col] = clip_extreme_values(df_complete[col])
    
    # Final check after cleaning
    check_for_problematic_values(df_complete, "after feature engineering and cleaning")
    
    print(f"Shape after feature engineering and dropping NAs: {df_complete.shape}")
    return df_complete

# Modify the train_and_evaluate_models function to handle errors
def train_and_evaluate_models(df, features):
    print("\nTraining and evaluating models...")
    # Prepare data for modeling
    X = df[features].copy()
    y = df['Target'].copy()
    
    # Use time series cross-validation
    tscv = TimeSeriesSplit(n_splits=5)
    
    # Define models to test
    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
        'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42, objective='reg:squarederror')
    }
    
    # Compare models
    results = {}
    for name, model in models.items():
        print(f"Evaluating {name}...")
        rmse_scores = []
        mae_scores = []
        r2_scores = []
        direction_accuracy = []
        
        for train_idx, test_idx in tscv.split(X):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            
            # Additional safety check for problematic values
            X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
            X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
            
            # Fill any remaining NaN with appropriate values
            X_train.fillna(X_train.median(), inplace=True)
            X_test.fillna(X_train.median(), inplace=True)  # Use training median for test data
            
            # Scale features with robust handling
            try:
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(X_train)
                X_test_scaled = scaler.transform(X_test)
                
                # Quick check for any remaining infinity values
                if np.any(np.isinf(X_train_scaled)) or np.any(np.isnan(X_train_scaled)):
                    print(f"Warning: Infinity or NaN values in X_train_scaled for {name}")
                    # Replace any remaining problematic values
                    X_train_scaled = np.nan_to_num(X_train_scaled, nan=0.0, posinf=0.0, neginf=0.0)
                
                if np.any(np.isinf(X_test_scaled)) or np.any(np.isnan(X_test_scaled)):
                    print(f"Warning: Infinity or NaN values in X_test_scaled for {name}")
                    # Replace any remaining problematic values
                    X_test_scaled = np.nan_to_num(X_test_scaled, nan=0.0, posinf=0.0, neginf=0.0)
                
                # Train model with error handling
                try:
                    model.fit(X_train_scaled, y_train)
                    
                    # Make predictions
                    y_pred = model.predict(X_test_scaled)
                    
                    # Calculate metrics
                    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
                    mae = mean_absolute_error(y_test, y_pred)
                    r2 = r2_score(y_test, y_pred)
                    
                    # Direction accuracy
                    actual_direction = np.sign(y_test.values - df['Close'].iloc[test_idx].values)
                    pred_direction = np.sign(y_pred - df['Close'].iloc[test_idx].values)
                    direction_acc = np.mean(actual_direction == pred_direction)
                    
                    rmse_scores.append(rmse)
                    mae_scores.append(mae)
                    r2_scores.append(r2)
                    direction_accuracy.append(direction_acc)
                    
                except Exception as e:
                    print(f"Error during model training/evaluation for {name}: {e}")
                    # Add placeholder values if model fails
                    rmse_scores.append(float('inf'))
                    mae_scores.append(float('inf'))
                    r2_scores.append(0)
                    direction_accuracy.append(0)
            
            except Exception as e:
                print(f"Error during scaling for {name}: {e}")
                # Add placeholder values if scaling fails
                rmse_scores.append(float('inf'))
                mae_scores.append(float('inf'))
                r2_scores.append(0)
                direction_accuracy.append(0)
        
        # Only calculate mean metrics if we have valid scores
        valid_scores = [score for score in rmse_scores if score != float('inf')]
        if valid_scores:
            results[name] = {
                'RMSE': np.mean(valid_scores),
                'MAE': np.mean([score for score in mae_scores if score != float('inf')]),
                'R²': np.mean([score for score in r2_scores if score != 0]),
                'Direction Accuracy': np.mean([score for score in direction_accuracy if score != 0])
            }
        else:
            print(f"Warning: No valid evaluation scores for {name}. Using placeholders.")
            results[name] = {
                'RMSE': float('inf'),
                'MAE': float('inf'),
                'R²': 0,
                'Direction Accuracy': 0
            }
    
    # Display results
    results_df = pd.DataFrame(results).T
    print("\nModel Comparison:")
    print(results_df)
    
    # Save results to CSV
    results_df.to_csv('model_comparison_results.csv')
    
    # Select best model based on direction accuracy and RMSE
    # First check if we have any valid models
    valid_models = results_df[results_df['RMSE'] < float('inf')].index.tolist()
    
    if not valid_models:
        print("No model succeeded in training. Using Linear Regression as fallback.")
        best_model_name = 'Linear Regression'
    else:
        best_model_name = results_df.loc[valid_models, 'Direction Accuracy'].idxmax()
        print(f"\nBest model based on Direction Accuracy: {best_model_name}")
        
        # In case there's a tie or very close scores, consider RMSE too
        if results_df.loc[best_model_name, 'RMSE'] > results_df.loc[valid_models, 'RMSE'].min() * 1.1:
            best_model_name = results_df.loc[valid_models, 'RMSE'].idxmin()
            print(f"Selecting model with lowest RMSE instead: {best_model_name}")
    
    return best_model_name, models[best_model_name], results_df


# 4. Feature Selection
def select_features(df):
    print("\nSelecting features...")
    # Prepare data for feature importance
    X = df.drop(['Target', 'Daily_Return'], axis=1)
    y = df['Target']
    
    # Use Random Forest to get feature importance
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)
    
    # Get feature importance
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    # Plot feature importance
    plt.figure(figsize=(12, 8))
    top_features = feature_importance.head(15)
    sns.barplot(x='Importance', y='Feature', data=top_features)
    plt.title('Top 15 Feature Importance')
    plt.tight_layout()
    plt.savefig('feature_importance.png')
    plt.close()
    
    print("Top 10 features:")
    print(feature_importance.head(10))
    
    # Select top features (you can adjust the number)
    top_features_list = feature_importance['Feature'].tolist()
    
    return top_features_list

# 5. Model Training and Evaluation
def train_and_evaluate_models(df, features):
    print("\nTraining and evaluating models...")
    # Prepare data for modeling
    X = df[features]
    y = df['Target']
    
    # Use time series cross-validation
    tscv = TimeSeriesSplit(n_splits=5)
    
    # Define models to test
    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
        'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42, objective='reg:squarederror')
    }
    
    # Compare models
    results = {}
    for name, model in models.items():
        print(f"Evaluating {name}...")
        rmse_scores = []
        mae_scores = []
        r2_scores = []
        direction_accuracy = []
        
        for train_idx, test_idx in tscv.split(X):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            
            # Scale features
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
            # Train model
            model.fit(X_train_scaled, y_train)
            
            # Make predictions
            y_pred = model.predict(X_test_scaled)
            
            # Calculate metrics
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            
            # Direction accuracy
            actual_direction = np.sign(y_test.values - df['Close'].iloc[test_idx].values)
            pred_direction = np.sign(y_pred - df['Close'].iloc[test_idx].values)
            direction_acc = np.mean(actual_direction == pred_direction)
            
            rmse_scores.append(rmse)
            mae_scores.append(mae)
            r2_scores.append(r2)
            direction_accuracy.append(direction_acc)
        
        results[name] = {
            'RMSE': np.mean(rmse_scores),
            'MAE': np.mean(mae_scores),
            'R²': np.mean(r2_scores),
            'Direction Accuracy': np.mean(direction_accuracy)
        }
    
    # Display results
    results_df = pd.DataFrame(results).T
    print("\nModel Comparison:")
    print(results_df)
    
    # Save results to CSV
    results_df.to_csv('model_comparison_results.csv')
    
    # Select best model based on direction accuracy and RMSE
    best_model_name = results_df['Direction Accuracy'].idxmax()
    print(f"\nBest model based on Direction Accuracy: {best_model_name}")
    
    # In case there's a tie or very close scores, consider RMSE too
    if results_df.loc[best_model_name, 'RMSE'] > results_df['RMSE'].min() * 1.1:
        best_model_name = results_df['RMSE'].idxmin()
        print(f"Selecting model with lowest RMSE instead: {best_model_name}")
    
    return best_model_name, models[best_model_name], results_df

# 6. Trading Strategy Evaluation
def evaluate_trading_strategy(df, model_name, model, features):
    print(f"\nEvaluating trading strategy with {model_name}...")
    
    # Create a copy of the dataframe for evaluation
    eval_df = df.copy()
    
    # Prepare features
    X = eval_df[features]
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Make predictions
    eval_df['Predicted_Price'] = model.predict(X_scaled)
    eval_df['Actual_Direction'] = np.sign(eval_df['Target'] - eval_df['Close'])
    eval_df['Predicted_Direction'] = np.sign(eval_df['Predicted_Price'] - eval_df['Close'])
    
    # Calculate accuracy
    eval_df['Correct_Prediction'] = (eval_df['Actual_Direction'] == eval_df['Predicted_Direction']).astype(int)
    direction_accuracy = eval_df['Correct_Prediction'].mean()
    print(f"Overall Direction Accuracy: {direction_accuracy:.4f}")
    
    # Simple trading strategy: Buy when predicted price is higher, sell when lower
    eval_df['Position'] = eval_df['Predicted_Direction'].shift(1)
    eval_df['Position'].fillna(0, inplace=True)
    
    # Calculate returns
    eval_df['Strategy_Return'] = eval_df['Position'] * eval_df['Daily_Return']
    
    # Calculate cumulative returns
    eval_df['Cumulative_Market_Return'] = (1 + eval_df['Daily_Return']).cumprod() - 1
    eval_df['Cumulative_Strategy_Return'] = (1 + eval_df['Strategy_Return']).cumprod() - 1
    
    # Calculate performance metrics
    total_return = eval_df['Cumulative_Strategy_Return'].iloc[-1]
    market_return = eval_df['Cumulative_Market_Return'].iloc[-1]
    
    # Calculate annualized return (assuming 252 trading days)
    years = len(eval_df) / 252
    annualized_return = (1 + total_return) ** (1 / years) - 1
    
    # Calculate Sharpe ratio (assuming 252 trading days)
    risk_free_rate = 0.02 / 252  # Assuming 2% annual risk-free rate
    sharpe_ratio = np.sqrt(252) * (eval_df['Strategy_Return'].mean() - risk_free_rate) / eval_df['Strategy_Return'].std()
    
    # Calculate maximum drawdown
    cum_returns = (1 + eval_df['Strategy_Return']).cumprod()
    running_max = cum_returns.cummax()
    drawdown = (cum_returns / running_max) - 1
    max_drawdown = drawdown.min()
    
    # Calculate win rate
    win_rate = (eval_df['Strategy_Return'] > 0).mean()
    
    # Plot cumulative returns
    plt.figure(figsize=(12, 6))
    plt.plot(eval_df.index, eval_df['Cumulative_Market_Return'], label='Buy and Hold')
    plt.plot(eval_df.index, eval_df['Cumulative_Strategy_Return'], label='Trading Strategy')
    plt.title('Cumulative Returns: Strategy vs Buy-and-Hold')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Return')
    plt.legend()
    plt.grid(True)
    plt.savefig('strategy_performance.png')
    plt.close()
    
    # Print performance metrics
    print("\nTrading Strategy Performance:")
    print(f"Total Return: {total_return:.4f}")
    print(f"Market Return: {market_return:.4f}")
    print(f"Annualized Return: {annualized_return:.4f}")
    print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
    print(f"Maximum Drawdown: {max_drawdown:.4f}")
    print(f"Win Rate: {win_rate:.4f}")
    
    # Create a dataframe with performance metrics
    performance = {
        'Total Return': total_return,
        'Market Return': market_return,
        'Annualized Return': annualized_return,
        'Sharpe Ratio': sharpe_ratio,
        'Maximum Drawdown': max_drawdown,
        'Win Rate': win_rate,
        'Direction Accuracy': direction_accuracy
    }
    
    performance_df = pd.DataFrame([performance])
    performance_df.to_csv('trading_strategy_performance.csv', index=False)
    
    return performance, eval_df

# 7. Final Model Training
def train_final_model(df, model_name, features):
    print(f"\nTraining final {model_name} model...")
    # Prepare data
    X = df[features]
    y = df['Target']
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Initialize the selected model
    if model_name == 'Linear Regression':
        final_model = LinearRegression()
    elif model_name == 'Random Forest':
        final_model = RandomForestRegressor(n_estimators=100, random_state=42)
    elif model_name == 'Gradient Boosting':
        final_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    elif model_name == 'XGBoost':
        final_model = xgb.XGBRegressor(n_estimators=100, random_state=42, objective='reg:squarederror')
    
    # Train the model
    final_model.fit(X_scaled, y)
    
    # Save model and scaler
    joblib.dump(final_model, 'stock_prediction_model.pkl')
    joblib.dump(scaler, 'feature_scaler.pkl')
    
    # Save feature list
    with open('selected_features.txt', 'w') as f:
        for feature in features:
            f.write(f"{feature}\n")
    
    return final_model, scaler
# Modify the generate_predictions function to work without test data
def generate_predictions(model, scaler, features, df, forecast_days=5):
    print("\nGenerating predictions for future days...")
    
    # Use the last available data to predict future prices
    latest_data = df.iloc[-1:].copy()
    
    # Create a dataframe for future dates
    future_dates = [latest_data.index[0] + pd.Timedelta(days=i) for i in range(1, forecast_days+1)]
    future_df = pd.DataFrame(index=future_dates)
    
    # Initialize with the latest values
    for col in df.columns:
        if col in ['Target']:
            continue
        future_df[col] = latest_data[col].values[0]
    
    # Make predictions day by day
    predictions = []
    
    for i in range(forecast_days):
        # Prepare features for the current day
        X = future_df.iloc[i:i+1][features]
        
        # Scale features
        X_scaled = scaler.transform(X)
        
        # Make prediction
        predicted_price = model.predict(X_scaled)[0]
        predictions.append(predicted_price)
        
        # Update the dataframe for the next day if needed
        if i < forecast_days - 1:
            # Update close price for the next day
            future_df.iloc[i+1, future_df.columns.get_loc('Close')] = predicted_price
            
            # Update other features based on the new close price
            # This is a simplified approach - in a real scenario, you would need
            # to update all relevant technical indicators
            if 'MA5' in future_df.columns:
                # Example: Update 5-day moving average
                ma5_values = list(df['Close'].iloc[-4:].values) + list(future_df['Close'].iloc[:i+1].values)
                future_df.iloc[i+1, future_df.columns.get_loc('MA5')] = sum(ma5_values) / 5
    
    # Create a dataframe with the predictions
    pred_df = pd.DataFrame({
        'Date': future_dates,
        'Predicted_Close': predictions
    })
    pred_df.set_index('Date', inplace=True)
    
    # Save predictions to CSV
    pred_df.to_csv('stock_predictions.csv')
    
    print(f"Predictions saved to 'stock_predictions.csv'")
    return pred_df

# 9. Write README file
def write_readme(model_name, performance, features):
    print("\nGenerating README file...")
    
    with open('README.md', 'w') as f:
        f.write("# Stock Price Prediction Model\n\n")
        
        f.write("## Overview\n")
        f.write("This project implements a machine learning model to predict stock prices 5 trading days into the future. ")
        f.write("The solution includes comprehensive data analysis, feature engineering, model selection, and trading strategy evaluation.\n\n")
        
        f.write("## Approach\n")
        f.write("1. **Data Preprocessing**: Handled missing values and prepared the time series data.\n")
        f.write("2. **Exploratory Data Analysis**: Analyzed trends, distributions, and correlations in the stock data.\n")
        f.write("3. **Feature Engineering**: Created technical indicators, moving averages, and other predictive features.\n")
        f.write("4. **Model Selection**: Compared multiple models including Linear Regression, Random Forest, Gradient Boosting, and XGBoost.\n")
        f.write("5. **Trading Strategy**: Implemented and evaluated a simulated trading strategy based on model predictions.\n\n")
        
        f.write(f"## Selected Model: {model_name}\n")
        f.write(f"The {model_name} model was selected based on its superior performance in both statistical accuracy and trading performance.\n\n")
        
        f.write("## Performance Metrics\n")
        f.write(f"- Total Return: {performance['Total Return']:.4f}\n")
        f.write(f"- Market Return: {performance['Market Return']:.4f}\n")
        f.write(f"- Annualized Return: {performance['Annualized Return']:.4f}\n")
        f.write(f"- Sharpe Ratio: {performance['Sharpe Ratio']:.4f}\n")
        f.write(f"- Maximum Drawdown: {performance['Maximum Drawdown']:.4f}\n")
        f.write(f"- Win Rate: {performance['Win Rate']:.4f}\n")
        f.write(f"- Direction Accuracy: {performance['Direction Accuracy']:.4f}\n\n")
        
        f.write("## Key Features\n")
        for i, feature in enumerate(features[:10], 1):
            f.write(f"{i}. {feature}\n")
        f.write("\n")
        
        f.write("## Limitations and Future Improvements\n")
        f.write("1. **Data Limitations**: The model relies solely on price and volume data. Incorporating fundamental data, market sentiment, and macroeconomic indicators could improve performance.\n")
        f.write("2. **Model Complexity**: More sophisticated models like LSTM networks or ensemble methods could potentially capture more complex patterns.\n")
        f.write("3. **Trading Strategy**: The current strategy is simplistic. More advanced position sizing, risk management, and portfolio optimization could enhance returns.\n")
        f.write("4. **Market Regimes**: The model doesn't explicitly account for different market regimes (bull, bear, sideways). Developing regime-specific models could be beneficial.\n\n")
        
        f.write("## Usage Instructions\n")
        f.write("1. Ensure all dependencies are installed: `pip install -r requirements.txt`\n")
        f.write("2. To make predictions on new data: `python predict.py --input new_data.csv`\n")
        f.write("3. Model files (`stock_prediction_model.pkl` and `feature_scaler.pkl`) are included for direct use.\n")
    
    print("README.md generated successfully")

# Modify the main function to use the updated generate_predictions function
def main():
    print("Stock Price Prediction Pipeline\n" + "="*30)
    try:
        # Define file path (update with your actual file path)
        file_path = '/kaggle/input/stock-data-analysis/question4-stock-data.csv'  
        
        # Step 1: Load and preprocess data
        df = load_and_preprocess_data(file_path)
        
        # Initial check for problematic values
        check_for_problematic_values(df, "after loading data")
        
        # Step 2: Perform EDA
        df = perform_eda(df)
        
        # Step 3: Engineer features
        df = engineer_features(df)
        
        # Replace any remaining infinite values
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df.dropna(inplace=True)
        
        # Final safety check
        if df.empty:
            raise ValueError("All data was filtered out during preprocessing. Check your dataset.")
        
        # Step 4: Select important features
        features = select_features(df)
        
        # Step 5: Train and evaluate models
        best_model_name, best_model, results = train_and_evaluate_models(df, features)
        
        # Step 6: Evaluate trading strategy
        performance, strategy_df = evaluate_trading_strategy(df, best_model_name, best_model, features)
        
        # Step 7: Train final model
        final_model, scaler = train_final_model(df, best_model_name, features)
        
        # Step 8: Generate predictions for future days
        predictions = generate_predictions(final_model, scaler, features, df)
        
        # Step 9: Write README
        write_readme(best_model_name, performance, features)
        
        print("\nStock prediction pipeline completed successfully!")
        print("Check the generated files for results and visualizations.")
        
    except Exception as e:
        print(f"Error in main pipeline: {e}")
        print("\nTroubleshooting suggestions:")
        print("1. Check your input data for missing values, zeros, or extreme outliers")
        print("2. Try preprocessing your data to handle outliers")
        print("3. Ensure your date column is properly formatted")
        print("4. If the error persists, you might need to modify feature engineering to avoid division by zero")

if __name__ == "__main__":
    main()

Stock Price Prediction Pipeline
Loading and preprocessing data...
Dataset shape: (11291, 6)
Missing values:
Adj Close     93
Close        117
High          95
Low          127
Open         103
Volume       145
dtype: int64
Missing values after handling:
Adj Close    0
Close        0
High         0
Low          0
Open         0
Volume       0
dtype: int64

Checking for problematic values at stage: after loading data
Total NaN values: 0
Total infinity values: 0

Performing Exploratory Data Analysis...
Augmented Dickey-Fuller Test:
ADF Statistic: -0.467371512452141
p-value: 0.8982279985700206

Engineering features...

Checking for problematic values at stage: after feature engineering, before cleaning
Total NaN values: 638
Columns with NaN values:
Daily_Return       1
MA5                4
MA10               9
MA20              19
MA50              49
MA200            199
RSI               13
MA20_std          19
Upper_Band        19
Lower_Band        19
BB_Width          19
Momentum      